In [13]:
#Script que cria a Tabela Loja


import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import random
import pyodbc
from datetime import date
from faker import Faker

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
#user = ""
#senha = ""

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
#conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

quant_clientes_gerados = 10

#Listas 

nomes_ruas = [
    "Rua Sao Joao", "Rua Santa Maria", "Rua Sao Paulo", "Rua Sao Pedro", "Rua Sao Jose",
    "Rua Sao Francisco", "Rua Santo Antonio", "Rua Sao Sebastiao", "Rua Santa Catarina",
    "Rua Luzia", "Rua Santa Rita", "Rua Santa Ana", "Rua Santa Terezinha", "Rua Santa Clara",
    "Rua Isabel", "Rua Santa Helena", "Rua Santa Cecilia", "Rua Santa Rosa", "Rua Santa Monica",
    "Rua Edwiges", "Rua Santa Ines", "Rua Santa Fe", "Rua Santa Lucia", "Rua Santa Efigenia",
    "Rua Barbara", "Rua Santa Angela", "Rua Santa Gertrudes", "Rua Santa Margarida",
    "Rua Marina", "Rua Santa Elisa", "Rua Santa Julia", "Rua Santa Zelia", "Rua Santa Zilda",
    "Rua Zulmira", "Rua Santa Zulene, Rua das Flores", "Rua da Paz", "Rua Sao Joao", "Rua Sao Paulo", "Rua Santo Antonio",
    "Rua 7 de Setembro", "Rua XV de Novembro", "Rua do Comercio", "Rua das Acacias", "Rua das Palmeiras",
    "Rua dos Cravos", "Rua do Sol", "Rua Lua Nova", "Rua da Esperanca", "Rua Sao Pedro",
    "Rua das Orquideas", "Rua das Rosas", "Rua Primavera", "Rua Santa Luzia", "Rua Sao Francisco",
    "Rua das Margaridas", "Rua do Bosque", "Rua das Aguas", "Rua Bela Vista", "Rua Santa Catarina",
    "Rua Dom Pedro I", "Rua Dom Pedro II", "Rua Marechal Deodoro", "Rua da Liberdade", "Rua do Lago",
    "Rua dos Girassois", "Rua Antonio Carlos", "Rua da Independencia", "Rua Presidente Vargas", "Rua das Laranjeiras",
    "Rua do Porto", "Rua Santos Dumont", "Rua Carlos Gomes", "Rua da Vitoria", "Rua do Carmo",
    "Rua do Campo", "Rua da Saudade", "Rua do Progresso", "Rua dos Bandeirantes", "Rua Coronel Antonio",
    "Rua Sao Jose", "Rua Benedito Calixto", "Rua dos Lirios", "Rua Sete Lagoas", "Rua Pedro Alvares Cabral",
    "Rua Castro Alves", "Rua Padre Anchieta", "Rua Machado de Assis", "Rua Monte Castelo", "Rua Humberto de Campos",
    "Rua dos Tucanos", "Rua das Violetas", "Rua do Ipe", "Rua dos Pinheiros", "Rua Jose Bonifacio",
    "Rua Getulio Vargas", "Rua Marechal Rondon", "Rua Visconde de Maua", "Rua do Horizonte", "Rua do Contorno",
    "Rua Rui Barbosa", "Rua Juscelino Kubitschek", "Rua Presidente Joao Goulart", "Rua General Osorio", "Rua dos Sonhos",
    "Rua das Figueiras", "Rua Goias", "Rua Alvorada", "Rua Bela Cintra", "Rua Piaui",
    "Rua Domingos Jorge Velho", "Rua General Carneiro", "Rua Dom Bosco", "Rua Miguel Couto", "Rua Alfredo Pujol",
    "Rua Frei Caneca", "Rua Dr. Arnaldo", "Rua Diogo Feijo", "Rua Regente Feijo", "Rua Conselheiro Lafaiete",
    "Rua Barao de Maua", "Rua Benjamin Constant", "Rua Santa Clara", "Rua Maranhao", "Rua Bahia",
    "Rua Amazonas", "Rua Acre", "Rua Sao Joaquim", "Rua dos Cedros", "Rua Tamoios",
    "Rua Jose de Alencar", "Rua Ceara", "Rua Camelia", "Rua das Cerejeiras", "Rua Japao"]

nomes_lojas = [
    "Centro", "Jardim", "Vila", "Sao Joao", "Sao Paulo", "Sao Pedro", "Sao Jose", "Sao Francisco",
    "Santo Antonio", "Sao Sebastiao", "Santa Maria", "Santa Catarina", "Santa Luzia", "Santa Rita",
    "Santa Ana", "Santa Terezinha", "Santa Clara", "Santa Isabel", "Santa Helena", "Santa Cecilia",
    "Santa Rosa", "Santa Monica", "Santa Edwiges", "Santa Ines", "Santa Fe", "Santa Lucia",
    "Santa Efigenia", "Santa Barbara", "Santa Angela", "Centro", "Jardim das Flores", "Vila Nova", "Jardim America", "Jardim Paulista",
    "Jardim Primavera", "Jardim Europa", "Jardim dos Ipes", "Jardim das Palmeiras", "Jardim Bela Vista",
    "Parque Industrial", "Jardim Alvorada", "Jardim Santa Clara", "Vila Esperanca", "Vila Maria",
    "Parque Sao Jorge", "Jardim Sao Francisco", "Jardim Santa Rita", "Vila Rica", "Vila Progresso",
    "Vila Matilde", "Vila Boa Vista", "Vila Sao Jose", "Parque dos Passaros", "Jardim Imperial",
    "Jardim Sao Luiz", "Jardim das Acacias", "Jardim Mariana", "Jardim do Lago", "Parque das Nacoes",
    "Parque dos Bandeirantes", "Jardim Atlantico", "Jardim Monte Castelo", "Jardim Eldorado", "Jardim Gloria",
    "Vila Oliveira", "Jardim Panorama", "Jardim Dom Bosco", "Vila Uniao", "Jardim Vitoria",
    "Jardim Novo Horizonte", "Vila Industrial", "Vila Augusta", "Jardim Botanico", "Jardim Sao Carlos",
    "Jardim Canada", "Jardim Santa Fe", "Jardim das Hortensias", "Parque Sao Bento", "Jardim dos Pinheiros",
    "Parque do Carmo", "Jardim Sao Joao", "Jardim Maria Helena", "Jardim Santa Rosa", "Jardim Felicidade",
    "Jardim Aurora", "Vila Santana", "Jardim Bela Vista", "Jardim das Magnolias", "Vila Medeiros",
    "Jardim dos Lirios", "Parque das Arvores", "Jardim Santa Terezinha", "Vila Nova Esperanca", "Jardim Santo Antonio",
    "Jardim Independencia", "Jardim Sao Jorge", "Jardim das Oliveiras", "Parque Estrela Dalva", "Jardim Mirante",
    "Vila Sao Pedro", "Parque Bela Vista", "Vila Nova Cachoeirinha", "Jardim Monte Alegre", "Parque Imperial",
    "Jardim Tropical", "Vila Santa Isabel", "Jardim Italia", "Jardim dos Cedros", "Jardim das Nacoes",
    "Vila Real", "Jardim dos Jacarandas", "Jardim Primavera", "Parque Paraiso", "Jardim Universo",
    "Jardim Santo Andre", "Jardim das Figueiras", "Vila Germanica", "Jardim Luz", "Vila Macedo",
    "Jardim Bonfim", "Jardim Colonial", "Jardim das Tulipas", "Vila Curuca", "Jardim Santa Maria"
]

cidades_brasil = [ 
    "Sao Paulo", "Rio de Janeiro", "Brasilia", "Salvador", "Fortaleza",
    "Belo Horizonte", "Manaus", "Curitiba", "Recife", "Goiania",
    "Porto Alegre", "Guarulhos", "Campinas", "Sao Luis",
    "Sao Goncalo", "Maceio", "Duque de Caxias", "Natal", "Teresina",
    "Sao Bernardo do Campo", "Joao Pessoa", "Osasco",
    "Jaboatao dos Guararapes", "Ribeirao Preto", "Uberlandia", "Contagem", "Sorocaba",
    "Aracaju", "Feira de Santana", "Cuiaba", "Joinville", "Aparecida de Goiania",
    "Londrina", "Juiz de Fora", "Ananindeua", "Belford Roxo", "Niteroi",
    "Sao Joao de Meriti", "Campos dos Goytacazes", "Serra", "Caxias do Sul", "Maua",
    "Macapa", "Piracicaba", "Canoas", "Bauru", "Sao Jose dos Campos"] #47 maiores cidades

df_maiores_cidades = df = pd.read_sql("SELECT IDCIDADE FROM CIDADE WHERE NOME IN ({})".format(','.join(f"'{cidade}'" for cidade in cidades_brasil)), engine)['IDCIDADE'].tolist()

# Função para gerar um nome completo, gênero e e-mail
def gerar_nome_lojas():
    nome = random.choice(nomes_lojas)
    numero = random.randint(10, 300)
    nomecompleto = f'Loja {nome} {numero}'
    rua = random.choice(nomes_ruas)
    num = random.randint(1000, 5000)
    bairro = random.choice(nomes_lojas)
    endereco = f"{rua} - {num} - {bairro}"
    id_cidade = random.choice(df_maiores_cidades)
    fake = Faker()
    data_criacao = fake.date_time_between(start_date='-15y',end_date='now')
    return nomecompleto, endereco, id_cidade, data_criacao

# Gerar 1000 nomes completos com gênero e e-mail
dados_nomes = [gerar_nome_lojas() for _ in range(quant_clientes_gerados)]
df_lojas = pd.DataFrame(dados_nomes, columns=['NOME', 'ENDERECO', 'ID_CIDADE', 'DATA_CRIACAO'])

df_lojas = df_lojas.drop_duplicates(subset=['NOME'])
df_lojas = df_lojas.apply(lambda col: col.str.replace(r'\s+', ' ', regex=True).str.strip() if col.dtype == 'object' and col.apply(lambda x: isinstance(x, str)).any() else col)

df_loja = pd.DataFrame({

                'NOME_LOJA':              df_lojas['NOME'],
                'ENDERECO':          df_lojas['ENDERECO'],
                'ID_CIDADE':         df_lojas['ID_CIDADE'],
                'DATA_CRIACAO':      df_lojas['DATA_CRIACAO'].dt.date,
                'DATA_ATLZ':         None,
                'VERSAO':            1,
})

display(df_loja)

tipo_dados_lojas = {

                'NOME_LOJA':              VARCHAR(100),
                'ENDERECO':          VARCHAR(100),
                'ID_CIDADE':         INTEGER,
                'DATA_CRIACAO':      DATE,
                'DATA_ATLZ':         DATE,
                'VERSAO':            INTEGER,
}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE LOJA"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_loja.to_sql(name='LOJA',con=engine,if_exists='append',index=False,dtype=tipo_dados_lojas)

engine.dispose()



Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida


,NOME_LOJA,ENDERECO,ID_CIDADE,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,Loja Santa Rosa 33,Rua Sao Jose - 3353 - Jardim das Flores,3221,2018-06-08,None,1
1,Loja Vila 112,Rua do Campo - 1422 - Santa Clara,3198,2022-10-01,None,1
2,Loja Sao Joao 78,Rua Jose Bonifacio - 1687 - Jardim Primavera,3165,2013-12-24,None,1
3,Loja Santo Antonio 111,Rua das Violetas - 2085 - Vila Boa Vista,5564,2023-08-19,None,1
4,Loja Santa Ana 103,Rua Amazonas - 1980 - Jardim Atlantico,3165,2012-05-04,None,1
5,Loja Vila Medeiros 72,Rua Sao Jose - 3099 - Jardim Santo Andre,2308,2014-06-25,None,1
6,Loja Parque das Arvores 280,Rua Santa Zelia - 3036 - Jardim dos Jacarandas,5564,2010-09-04,None,1
7,Loja Vila Macedo 71,Rua Padre Anchieta - 1251 - Santa Efigenia,4446,2024-02-03,None,1
8,Loja Jardim Botanico 86,Rua das Rosas - 2080 - Jardim Sao Jorge,3753,2020-02-26,None,1
9,Loja Sao Sebastiao 86,Rua Sao Francisco - 2543 - Vila Boa Vista,3753,2016-11-03,None,1
